In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import numpy as np
import statsmodels.api as sm
from sklearn.cluster import KMeans
#import scikit_posthocs as sp
from scipy.stats import kruskal
from sklearn.utils import resample
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

pd.set_option('display.max_columns', None)  # Отображение всех столбцов
# pd.set_option('display.max_rows', None)  # Ото----бражение всех строк   

regions = pd.read_csv('regions.csv')
starts = pd.read_csv('starts.csv')
volunteers = pd.read_csv('volunteers.csv')


volunteers = volunteers[volunteers.athlete_age < 100]
starts = starts[starts.athlete_age < 80]
starts = starts[starts.finish_time < 5000]

starts['rain.1h'] = starts['rain.1h'].fillna(0)
starts['snow.1h'] = starts['snow.1h'].fillna(0)

cnt_ludei = starts.groupby(['event_name', 'event_date', 'region']).agg(
    cnt_runners = ('athlete_id', 'count')
)

ages_14 = starts[starts['athlete_age'] == 14]['athlete_id'].unique()
ages_15 = starts[starts['athlete_age'] == 15]['athlete_id'].unique()
common_athletes = set(ages_14) & set(ages_15)
starts = starts[~starts['athlete_id'].isin(common_athletes)]



starts['is_male'] = starts['athlete_gender'].apply(lambda x: 1 if x == 'мужской' else 0)
starts = starts[starts['athlete_age'] > 14]

volunteer_counts = volunteers.groupby(['event_date','event_name', 'volunteer_role_name']).size().unstack(fill_value=0)  
starts = starts.merge(volunteer_counts, on=['event_date','event_name'], how='left')  

starts = starts.dropna(subset=[col for col in starts.columns if col not in ['snow.1h', 'rain.1h']])   # удаление где 0 волонтеров в сумме было


unique_dates = (
    pd.concat([
        starts[['athlete_id', 'event_date']],
        volunteers[['athlete_id', 'event_date']]
    ])
    .drop_duplicates()
)
date_counts = unique_dates.groupby('athlete_id').size().reset_index(name='unique_event_count')
date_counts['is_repeated'] = (date_counts['unique_event_count'] > 1).astype(int)
starts = starts.merge(date_counts[['athlete_id', 'is_repeated']], on='athlete_id', how='left')


# starts = starts[~starts['region'].isin(['Республика Адыгея', 'Республика Карелия', 'Липецкая область', 'Сахалинская область', 'Архангельская область'])]  
starts = starts[starts.event_date <= '2023-11-15']


newbie = starts[starts['is_newbie'] == 1]
first_runs = newbie.sort_values(by='event_date').drop_duplicates('athlete_id')
didnt_come = first_runs[first_runs.is_repeated == 0]
come = first_runs[first_runs.is_repeated == 1]
come_with_didnt_come = pd.concat([come, didnt_come])
come_with_didnt_come['cnt_volunteers'] = come_with_didnt_come.iloc[:, 25:52].sum(axis=1)
come_with_didnt_come = come_with_didnt_come.merge(cnt_ludei,on=['event_name', 'event_date', 'region'], how="left")
print(len(come_with_didnt_come))

18114


In [9]:
# Словарь соответствия регионов федеральным округам
region_to_district = {
    # Центральный федеральный округ
    'Белгородская область': 'Центральный',
    'Владимирская область': 'Центральный',
    'Воронежская область': 'Центральный',
    'Калужская область': 'Центральный',
    'Курская область': 'Центральный',
    'Липецкая область': 'Центральный',
    'Московская область': 'Центральный',
    'Смоленская область': 'Центральный',
    'Тамбовская область': 'Центральный',
    'Тульская область': 'Центральный',
    'Ярославская область': 'Центральный',
    'Москва': 'Центральный',

    # Северо-Западный федеральный округ
    'Архангельская область': 'Северо-Западный',
    'Калининградская область': 'Северо-Западный',
    'Ленинградская область': 'Северо-Западный',
    'Новгородская область': 'Северо-Западный',
    'Псковская область': 'Северо-Западный',
    'Санкт-Петербург': 'Северо-Западный',
    'Республика Коми': 'Северо-Западный',

    # Южный федеральный округ
    'Астраханская область': 'Южный',
    'Волгоградская область': 'Южный',
    'Краснодарский край': 'Южный',
    'Ростовская область': 'Южный',

    # Северо-Кавказский федеральный округ
    'Ставропольский край': 'Северо-Кавказский',
    'Республика Северная Осетия — Алания': 'Северо-Кавказский',

    # Приволжский федеральный округ
    'Кировская область': 'Приволжский',
    'Нижегородская область': 'Приволжский',
    'Оренбургская область': 'Приволжский',
    'Пермский край': 'Приволжский',
    'Самарская область': 'Приволжский',
    'Республика Башкортостан': 'Приволжский',
    'Республика Марий Эл': 'Приволжский',
    'Республика Татарстан': 'Приволжский',
    'Удмуртская Республика': 'Приволжский',
    'Чувашская Республика': 'Приволжский',

    # Уральский федеральный округ
    'Курганская область': 'Уральский',
    'Свердловская область': 'Уральский',
    'Тюменская область': 'Уральский',
    'Челябинская область': 'Уральский',

    # Сибирский федеральный округ
    'Иркутская область': 'Сибирский',
    'Кемеровская область': 'Сибирский',
    'Новосибирская область': 'Сибирский',
    'Омская область': 'Сибирский',
    'Томская область': 'Сибирский',
    'Красноярский край': 'Сибирский',
    'Республика Саха (Якутия)': 'Сибирский',

    # Дальневосточный федеральный округ
    'Сахалинская область': 'Дальневосточный',
}

come_with_didnt_come['federal_district'] = come_with_didnt_come['region'].map(region_to_district)


In [18]:
come_with_didnt_come.federal_district.value_counts()

federal_district
Центральный          6211
Уральский            3897
Северо-Западный      2279
Приволжский          2139
Сибирский            1541
Южный                1302
Северо-Кавказский     680
Дальневосточный        65
Name: count, dtype: int64

Строим для центрального округа

In [12]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46) & (come_with_didnt_come.federal_district == 'Центральный')]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46) & (come_with_didnt_come.federal_district == 'Центральный')]
print(len(come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46) & (come_with_didnt_come.federal_district == 'Центральный')].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

1350
Optimization terminated successfully.
         Current function value: 0.658367
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 3879
Model:                          Logit   Df Residuals:                     3851
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.02820
Time:                        00:17:12   Log-Likelihood:                -2553.8
converged:                       True   LL-Null:                       -2627.9
Covariance Type:            nonrobust   LLR p-value:                 1.079e-18
                                       coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------

Значимо маршал и инструктаж

In [14]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 47) & (come_with_didnt_come.cnt_runners <= 138) & (come_with_didnt_come.federal_district == 'Центральный')]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 47) & (come_with_didnt_come.cnt_runners <= 138) & (come_with_didnt_come.federal_district == 'Центральный')]

print(len(come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 47) & (come_with_didnt_come.cnt_runners <= 138) & (come_with_didnt_come.federal_district == 'Центральный')].drop_duplicates(['event_name', 'event_date'])))

need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 47-138 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

333
Optimization terminated successfully.
         Current function value: 0.663940
         Iterations 5


Модель для предсказывания is_repeated новичка, в пробежках где было 47-138 людей


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 2120
Model:                          Logit   Df Residuals:                     2092
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.03177
Time:                        00:19:27   Log-Likelihood:                -1407.6
converged:                       True   LL-Null:                       -1453.7
Covariance Type:            nonrobust   LLR p-value:                 4.456e-09
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------

никто не значим

In [13]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 139) & (come_with_didnt_come.cnt_runners <= 404) & (come_with_didnt_come.federal_district == 'Центральный')]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 139) & (come_with_didnt_come.cnt_runners <= 404) & (come_with_didnt_come.federal_district == 'Центральный')]
print(len(come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 139) & (come_with_didnt_come.cnt_runners <= 404) & (come_with_didnt_come.federal_district == 'Центральный')].drop_duplicates(['event_name', 'event_date'])))



need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 139-404 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

6
Возможна мультиколлинеарность:  -0.8498270909934158 feels_like Маршал


/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


Возможна мультиколлинеарность:  0.8331206158317529 pressure Маршал
Возможна мультиколлинеарность:  0.8495032803905707 pressure Фотограф
Возможна мультиколлинеарность:  0.8264154447257516 pressure Хранение и доставка оборудования
Возможна мультиколлинеарность:  -0.9337356905430894 humidity wind_speed
Возможна мультиколлинеарность:  -0.8163096273832613 humidity Проверка трассы
Возможна мультиколлинеарность:  -0.9337356905430894 wind_speed humidity
Возможна мультиколлинеарность:  0.825834825844406 wind_speed Проверка трассы
Возможна мультиколлинеарность:  0.8780541003633613 Ведущий велосипед cnt_runners
Возможна мультиколлинеарность:  -0.8498270909934158 Маршал feels_like
Возможна мультиколлинеарность:  0.833120615831753 Маршал pressure
Возможна мультиколлинеарность:  0.9368843975586921 Маршал Фотограф
Возможна мультиколлинеарность:  0.8559785618124017 Маршал Хранение и доставка оборудования
Возможна мультиколлинеарность:  0.8213610510318569 Подготовка мероприятия Разное
Возможна мультико

LinAlgError: Singular matrix

мало записей

In [15]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46) & (come_with_didnt_come.federal_district == 'Северо-Западный')]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46) & (come_with_didnt_come.federal_district == 'Северо-Западный')]
print(len(come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46) & (come_with_didnt_come.federal_district == 'Северо-Западный')].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

338
Optimization terminated successfully.
         Current function value: 0.646826
         Iterations 24


/opt/anaconda3/lib/python3.11/site-packages/statsmodels/discrete/discrete_model.py:2383: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


LinAlgError: Singular matrix

мало записей

In [16]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 46) & (come_with_didnt_come.cnt_runners <= 138) & (come_with_didnt_come.federal_district == 'Северо-Западный')]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 46) & (come_with_didnt_come.cnt_runners <= 138) & (come_with_didnt_come.federal_district == 'Северо-Западный')]
print(len(come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 46) & (come_with_didnt_come.cnt_runners <= 138) & (come_with_didnt_come.federal_district == 'Северо-Западный')].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

152


/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


Optimization terminated successfully.
         Current function value: 0.658461
         Iterations 6


LinAlgError: Singular matrix

мало записей

In [19]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46) & (come_with_didnt_come.federal_district == 'Уральский')]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46) & (come_with_didnt_come.federal_district == 'Уральский')]
print(len(come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 2) & (come_with_didnt_come.cnt_runners <= 46) & (come_with_didnt_come.federal_district == 'Уральский')].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

167


/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


Optimization terminated successfully.
         Current function value: 0.611533
         Iterations 6


Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                  505
Model:                          Logit   Df Residuals:                      478
Method:                           MLE   Df Model:                           26
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                  0.1026
Time:                        00:32:37   Log-Likelihood:                -308.82
converged:                       True   LL-Null:                       -344.15
Covariance Type:            nonrobust   LLR p-value:                 5.345e-06
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------

Сканирование штрих кодов значимость

In [20]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 46) & (come_with_didnt_come.cnt_runners <= 138) & (come_with_didnt_come.federal_district == 'Уральский')]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 46) & (come_with_didnt_come.cnt_runners <= 138) & (come_with_didnt_come.federal_district == 'Уральский')]
print(len(come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 46) & (come_with_didnt_come.cnt_runners <= 138) & (come_with_didnt_come.federal_district == 'Уральский')].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

104
Optimization terminated successfully.
         Current function value: 0.641088
         Iterations 6


Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей


                           Logit Regression Results                           
Dep. Variable:            is_repeated   No. Observations:                 1096
Model:                          Logit   Df Residuals:                     1068
Method:                           MLE   Df Model:                           27
Date:                Sun, 15 Dec 2024   Pseudo R-squ.:                 0.06170
Time:                        00:40:41   Log-Likelihood:                -702.63
converged:                       True   LL-Null:                       -748.83
Covariance Type:            nonrobust   LLR p-value:                 4.390e-09
                                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------

Сканирование штрих кодов, пейсер

In [21]:
X = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 138) & (come_with_didnt_come.cnt_runners <= 404) & (come_with_didnt_come.federal_district == 'Уральский')]
y = come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 138) & (come_with_didnt_come.cnt_runners <= 404) & (come_with_didnt_come.federal_district == 'Уральский')]
print(len(come_with_didnt_come[(come_with_didnt_come.cnt_runners >= 138) & (come_with_didnt_come.cnt_runners <= 404) & (come_with_didnt_come.federal_district == 'Уральский')].drop_duplicates(['event_name', 'event_date'])))


need_columns = ['finish_time', 'athlete_age', 'feels_like', 'pressure', 'humidity', 'wind_speed', 
                'Ведущий велосипед', 
                'Завершение мероприятия', 'Замыкающий', 'Инструктаж новых участников', 'Маршал', 'Обработка результатов', 'Организатор',
                'Организация финиша', 'Пейсер', 'Подготовка мероприятия', 
            'Проверка карточек позиций', 'Проверка трассы', 'Раздача карточек позиций',
                'Разное', 'Связи с общественностью', 'Секундомер', 'Сканирование штрих-кодов', 'Сортировка карточек',
                'Фотограф', 'Хранение и доставка оборудования',
                  'cnt_runners'
                  ]
                






for col1 in need_columns:
    for col2 in need_columns:
        if col1 != col2:
            try:
                coef = X[col1].corr(X[col2],method='spearman')
                if abs(coef) > 0.8:
                    print('Возможна мультиколлинеарность: ', coef, col1, col2)
            except:
                print('ошибка в цикле на проверке корреляций между ', col1, " ", col2)

for_model = X[need_columns]


for_model = sm.add_constant(for_model)
y = y['is_repeated']
model = sm.Logit(y, for_model).fit()
print('\n')
print('Модель для предсказывания is_repeated новичка, в пробежках где было 2-46 людей')
print('\n')
print(model.summary())
marginal_effects = model.get_margeff()
print('\n')
print('Предельные эффекты:')
print(marginal_effects.summary())
print('\n')

y_pred = model.predict(for_model)
y_pred_class = (y_pred > 0.5).astype(int)  
cm = confusion_matrix(y, y_pred_class)
print(cm)
accuracy = (y_pred_class == y).mean()  
print(f'Accuracy: {accuracy:.2f}')  

41


/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


Optimization terminated successfully.
         Current function value: 0.648587
         Iterations 5


LinAlgError: Singular matrix